In [1]:
from tkinter import *
from PIL import Image, ImageTk
#from tkinter import filedialog
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
#import random
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.decomposition import PCA
import glob

In [2]:
pickle_in = open('X.pickle','rb')
X = pickle.load(pickle_in)
pickle_in = open('y.pickle','rb')
y = pickle.load(pickle_in)
pickle_in = open('Z.pickle','rb')
Z = pickle.load(pickle_in)

In [3]:
X_train, X_test, y_train, y_test, Z_train, Z_test = train_test_split(X, y, Z, stratify=y, random_state=0)
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [4]:
def recortar(imagen):
    for im in glob.glob("Usuario/*.jpg"):
        try:
            os.remove(im)
        except OSError as e:
            print(f"Error:{ e.strerror}")   
    image = imagen
    gradX = cv2.Sobel(image, cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(image, cv2.CV_32F, dx=0, dy=1, ksize=-1)
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)
    blurred = cv2.blur(gradient, (9, 9))
    _, thresh = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    closed = cv2.erode(closed, None, iterations=4)
    closed = cv2.dilate(closed, None, iterations=4)
    a=100
    c=0
    Imagen=closed
    while a>2:
        (cntsf, xf) = cv2.findContours(Imagen.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cf = sorted(cntsf, key=cv2.contourArea, reverse=True)[0]
        rectf = cv2.minAreaRect(cf)
        boxf = np.int0(cv2.boxPoints(rectf))
        Xsf = [i[0] for i in boxf]
        Ysf = [i[1] for i in boxf]
        x1f = min(Xsf)
        x2f = max(Xsf)
        y1f = min(Ysf)
        y2f = max(Ysf)
        cropImgf= image[y1f:y2f, x1f:x2f]
        #plt.imshow(cropImgf)
        A=str(c)
        cv2.imwrite("Usuario/imagen"+A+".jpg", cropImgf)
        #HASTA AQUI SE HIZO TODO UNA VEZ
        b=0
        dimensionesf = (closed.shape[0],closed.shape[1])
        arrayf=np.zeros(dimensionesf)
        for i in range(closed.shape[1]):
            for j in range(closed.shape[0]):
                if (x1f < i < x2f and y1f < j < y2f):
                    arrayf[j,i]=0
                else:
                    arrayf[j,i]=Imagen[j,i]
        for i in range(closed.shape[1]):
            for j in range(closed.shape[0]):
                if (arrayf[j,i]>0):
                    b=b+1
        a=(b*100)/(closed.shape[0]*closed.shape[1])
        c=c+1
        Imagen= np.uint8(arrayf)

In [5]:
def clase(i):
    n=''
    if (i==0):
        n='Dado'
    if (i==1):
        n='Dardo'
    if (i==2):
        n='Foco'
    if (i==3):
        n='Maceta'
    if (i==4):
        n='Oso'
    if (i==5):
        n='Pulpo'
    if (i==6):
        n='Rueda'
    return n

In [6]:
def predecir(imagen):
    img = cv2.imread(imagen, cv2.IMREAD_GRAYSCALE)
    img_N = cv2.resize(img,(140,140))
    img_V = (np.asarray(img_N)).flatten()
    prediccion = knn.predict([img_V])
    for i in range(7):
        if (prediccion==i):
            p=clase(i)
    return(p)       

In [7]:
img = cv2.imread('pulpo.png', cv2.IMREAD_GRAYSCALE)

In [ ]:
recortar(img)

In [ ]:
Numero=len(glob.glob("Usuario/*.jpg"))
for i in range(Numero):
    I=str(i)
    exec('img_{}=predecir("Usuario/imagen"+I+".jpg")'.format(i))
#for i in range(Numero):
    exec('print(img_{})'.format(i))


In [ ]:
img_0